# FNN Linus Etemi

## Imports


In [3]:
TYPE = 'GPU'

if TYPE == 'CPU':
    import numpy as np
else:
    import cupy as np
    
import scipy.special
import math
import matplotlib
import matplotlib.pyplot as matplot
import pickle

from ray import tune
from csv import writer


## Config

In [4]:
#hiddennodes_start = 800
#hiddennodes_stop = 800
#hiddennodes_step = 100

#learningrate_start = 0.08
#learningrate_stop = 0.08
#learningrate_step = 0.01

#epochs_start = 1
#epochs_stop = 2
#epochs_step = 0

outputnodes = 10
hiddennodes = 900
learningrate = 0.05
inputnodes = 784
epochs= 1


## Neuronales Netz 

In [5]:
class neuralNetwork:
    def __init__(self, innodes, outnodes, hidnodes, learningrate, epochs):
        self.inodes = innodes
        self.onodes = outnodes
        self.hnodes = hidnodes
        self.lr = learningrate
        self.epo = epochs
        self.wih = ((np.random.rand(self.hnodes,self.inodes)*2)-1)
        self.who = ((np.random.rand(self.onodes,self.hnodes)*2)-1)
        pass
    
    def activation_function(self, z: int) -> classmethod:
        return 1 / (1 + np.exp(-z))

    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)

        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors) 
        self.who += self.lr * np.dot( (output_errors*final_outputs*(1.0-final_outputs)), np.transpose(hidden_outputs)) 
        self.wih += self.lr * np.dot( (hidden_errors*hidden_outputs*(1.0-hidden_outputs)), np.transpose(inputs)) 
        pass

    def query(self, input_list):
        inputs = np.array(input_list, ndmin=2).T
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    

    

In [6]:
def clear_content(fname: str) -> None:
    file = open(fname, "w+")
    file.close

## Train_Network

In [7]:
def train_network(n, net, epo):  

        clear_content('mnist_second_train.csv')
        training_data_file = open("./MNISTData_red/mnist_train_red_8_9_by_90P.csv", 'r')
        training_data_list = training_data_file.readlines()
        training_data_file.close()

        for e in range(epo):
            for record in training_data_list:
                all_values = record.split(',')

                inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01
                targets = np.zeros(outputnodes) + 0.01
                targets[int(all_values[0])] = 0.99
                n.train(inputs, targets)

                pass
            pass



## Train_Network 2


In [8]:
# def train_network(n, net, epo):  
#         if(net == 1):
#             n.clear_content('mnist_second_train.csv')
#             training_data_file = open("./MNISTData_red/mnist_train_red_8_9_by_90P.csv", 'r')
#             training_data_list = training_data_file.readlines()
#             training_data_file.close()
#         elif(net == 2):
#             training_data_file = open("./mnist_second_train.csv", 'r')
#             training_data_list = training_data_file.readlines()
#             training_data_file.close()
#         for e in range(epo):
#             for record in training_data_list:
#                 all_values = record.split(',')

#                 inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01
#                 targets = np.zeros(outputnodes) + 0.01
#                 targets[int(all_values[0])] = 0.99
#                 n.train(inputs, targets)

#                 pass
#             pass


## Test_Network

In [9]:
def test_network(n):

    test_data_file = open("./MNISTData_red/mnist_train_red_8_9_by_90P.csv", 'r')
    test_data_list = test_data_file.readlines()
    test_data_file.close()
    array_data_all = []
    scorecard_p = []    
    scorecard_q = []

    for i in range(100):
        array_data_all.append([])
        scorecard_p.append([])
        scorecard_q.append([])

    for record in test_data_list:
        all_values = record.split(',')
        correct_label = int(all_values[0])
        inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01
        outputs = n.query(inputs)
        label_max = outputs[np.argmax(outputs)]
        label = np.argmax(outputs)

        for threshold in range(100):
            if(label_max > (threshold/100)):
                scorecard_p[threshold].append(1)
                if(label == correct_label):
                    scorecard_q[threshold].append(1)
                else:
                    scorecard_q[threshold].append(0)
            else:
                scorecard_p[threshold].append(0)
                #zweite netz werte 

    for i in range(100):
        scorecard_p_array = np.asarray(scorecard_p[i])
        scorecard_q_array = np.asarray(scorecard_q[i])

        array_data_all[i].append(i/100)
        array_data_all[i].append('P: {:.5f}'.format(scorecard_p_array.sum()/scorecard_p_array.size))
        array_data_all[i].append('Q: {:.5f}'.format(scorecard_q_array.sum()/scorecard_q_array.size))


    return array_data_all

## Test2

In [10]:
#def test_network2(n):
    # test_data_file = open("./mnist_accepted.csv", 'r')
    # test_data_list = test_data_file.readlines()
    # test_data_file.close()

    # scorecard_train = []

    # for record in test_data_list:

    #         all_values = record.split(',')
    #         correct_label = int(all_values[0])
    #         inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01
    #         outputs = n.query(inputs)
    #         score = outputs[np.argmax(outputs)]

    #         if(np.argmax(outputs) == correct_label):
    #             scorecard_train.append(1)
    #         else:
    #             scorecard_train.append(0)
    #             append_list_as_row('mnist_second_train.csv', list(map(int, all_values)))
                    


    # scorecard_train_array = np.asarray(scorecard_train)
    # performance2 = scorecard_train_array.sum() / scorecard_train_array.size

    # return performance2

## CSV Writer und Grid Search

In [11]:
def append_list_as_row(fname,list_of_elem):
    with open(fname, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [12]:
#learningrate_i = learningrate_start
#hiddennodes_i = hiddennodes_start
#epochs_i = epochs_start


#while epochs_i <= epochs_stop:
#    while hiddennodes_i <= hiddennodes_stop:
#        while learningrate_i <= learningrate_stop:
#            intermediate_score = neuralNetwork(784,10,hiddennodes_i,learningrate_i,epochs_i).test_train()
#            print('Epoch [{:02d}/{:02d}]     | Hidden_Nodes [{:04d}/{:04d}]     | Learning_Rate [{:.2f}/{:.2f}]     | Performance: {:.5f}'.format(epochs_i,epochs_stop,hiddennodes_i,hiddennodes_stop,learningrate_i,learningrate_stop,intermediate_score))
#            append_list_as_row([784,10,hiddennodes_i,learningrate_i,epochs_i,intermediate_score])
#            learningrate_i += learningrate_step 
#        learningrate_i = learningrate_start    
#        hiddennodes_i += hiddennodes_step
#        print("========================================================")
#    hiddennodes_i = hiddennodes_start
#    epochs_i += epochs_step
#    print("========================================================")

## Scoring und Aufruf von Beiden Netzen

In [13]:

n1 = neuralNetwork(784,10,hiddennodes,learningrate,epochs)
#train_network(n1, network_number, epochs)
#pickle.dump(n1, open('neural_network_1.obj', 'wb'))
n1 = pickle.load(open('neural_network_1.obj', 'rb'))

intermediate_score1 = test_network(n1)
print(intermediate_score1)


# network_number=2
# n2 = neuralNetwork(784,10,hiddennodes,learningrate,epochs)
# train_network(n2, network_number, epochs)
# intermediate_score2 = test_network(n2, network_number)
# print(intermediate_score2)


[[0.0, 1, 1], [0.01, 1, 1], [0.02, 1, 1], [0.03, 1, 1], [0.04, 1, 1], [0.05, 1, 1], [0.06, 1, 1], [0.07, 1, 1], [0.08, 1, 1], [0.09, 1, 1], [0.1, 1, 1], [0.11, 1, 1], [0.12, 1, 1], [0.13, 1, 1], [0.14, 1, 1], [0.15, 1, 1], [0.16, 1, 1], [0.17, 1, 1], [0.18, 1, 1], [0.19, 1, 1], [0.2, 1, 1], [0.21, 1, 1], [0.22, 1, 1], [0.23, 1, 1], [0.24, 1, 1], [0.25, 1, 1], [0.26, 1, 1], [0.27, 1, 1], [0.28, 1, 1], [0.29, 1, 1], [0.3, 1, 1], [0.31, 1, 1], [0.32, 1, 1], [0.33, 1, 1], [0.34, 1, 1], [0.35, 1, 1], [0.36, 1, 1], [0.37, 1, 1], [0.38, 1, 1], [0.39, 1, 1], [0.4, 1, 1], [0.41, 1, 1], [0.42, 1, 1], [0.43, 1, 1], [0.44, 1, 1], [0.45, 1, 1], [0.46, 1, 1], [0.47, 1, 1], [0.48, 1, 1], [0.49, 1, 1], [0.5, 1, 1], [0.51, 1, 1], [0.52, 1, 1], [0.53, 1, 1], [0.54, 1, 1], [0.55, 1, 1], [0.56, 1, 1], [0.57, 1, 1], [0.58, 1, 1], [0.59, 1, 1], [0.6, 1, 1], [0.61, 1, 1], [0.62, 1, 1], [0.63, 1, 1], [0.64, 1, 1], [0.65, 1, 1], [0.66, 1, 1], [0.67, 1, 1], [0.68, 1, 1], [0.69, 1, 1], [0.7, 1, 1], [0.71, 1, 1],

## Plotting

In [14]:
l = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
t_values = []
p_values = []
q_values = []
for i in range(100):
    speicher = intermediate_score1[i]
    t_values.append(speicher[0])
    p_values.append(speicher[1])
    q_values.append(speicher[2])
    